In [2]:
import pandas as pd
import os
import json
import plotly 
import plotly.plotly as py
from plotly.graph_objs import *
import numpy as np
import plotly.plotly as py
import plotly.graph_objs as graph_objs

In [3]:
#Initial set up
#1. set up data dir to read data files
dir = os.getcwd()+'/data'
#2.set up plotly credentials and mapbox access_token in order to connect plotly with mapbox
plotly.tools.set_credentials_file(username='StellaSun', api_key='gmyMYPK8gH2mJVr1t7Fj')
mapbox_access_token = 'pk.eyJ1Ijoic3N1bjEwIiwiYSI6ImNqYThqNmZjMTAwdDEycXFybTBxNGUxaWcifQ.BL82FAy1_HdKC5DCNrieCg'

In [5]:
#read location data to make the simple scatter plot on mapbox
df = pd.read_csv(dir + '/pizza_locations_google.csv')
for col in df.columns:
    df[col] = df[col].astype(str)
    
lat = df.lat
lng = df.lng
df['text'] = df['name'] + '<br>' +\
       'Price Range:' + df['price_level'] + '<br>' +\
       'Rating:' + df['rating']

In [6]:
#make first plot to show individual pizza store location with some information
data = Data([
    Scattermapbox(
        lat=lat,
        lon=lng,
        mode='markers',
        marker=Marker(
            size=6,
            color='rgb(84,39,143)',
            opacity=0.7,
        ),
        text=df.text,
        hoverinfo='text',
        hoverlabel=dict(
            bgcolor='rgb(255,255,255)',
            font=dict(
            family='Times New Roman',
            color = 'rgb(84,39,143)')
            )
    )
])

#set up manhattan view
layout = Layout(
    autosize=True,
    hovermode='closest',
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=dict(
            lat=40.75,
            lon=-73.9
        ),
        pitch=0,
        zoom=10
    ),
)

fig = dict(data=data, layout=layout)
py.iplot(fig, filename='Manhattan')

In [7]:
#in order to make a choropleth map, we download the geojson data to show zipcode 
data = json.load(open(dir+'/Json/Manhattan ZipCode.geojson'))
#read local file that contains distance of pizza stores within all manhattan zipcodes
distance_df = pd.read_csv(dir+'/Zipcode_Avg_Dist.csv')
distance_df = distance_df.drop('place_id',axis = 1)
for i in distance_df.columns:
    distance_df = distance_df.rename(columns={i:i.split(".")[0]})

In [8]:
#since there is no 10129 zipcode in the raw geojson. We deleted from our distance file
distance_df = distance_df.drop('10129',axis=1)

In [9]:
df = distance_df[-1:]
df = df*0.1
distance = list(df.loc[153])
zipcode = df.columns

In [10]:
#create a dictionary that has the zipcode as key, and coordinated boundary as values
zipcode_boudary = dict()
#filter the geojson data that only has manhattan zipcode
for i in data['features']:
    if i['properties']['borough'] == 'Manhattan':
        if i['properties']['postalCode'] in zipcode:
            zipcode_boudary[i['properties']['postalCode']] = i['geometry']

In [11]:
#format the distance to 1 decimal point
if (len(zipcode_boudary.keys()) == len(zipcode)):
    formattedList = ["%.1f" % member for member in distance]
    test = dict(zip(zipcode,formattedList))
else:
    print("Double Check")

In [12]:
#Generate new geojson data with distance range and write into local folder
for i in zipcode:
    name = 'data_'+ i
    result = {"type": "FeatureCollection"}
    featurelist = list()
    feat = dict()
    feat['type'] = "Feature"
    feat['properties']={'postcode':i}
    feat['geometry'] = zipcode_boudary[i]
    featurelist.append(feat)
    result['features'] = featurelist
    
    with open(dir+'/Json/'+name+'.json', 'w') as f:
        f.write(json.dumps(result))

In [13]:
#make a dictionary for distance range and color range 
dist_range = np.arange(0,1.1,0.1)
dist_range = ["%.1f" % member for member in dist_range]
    
colorlist = ['rgb(128,0,38)','rgb(189,0,38)','rgb(227,26,28)','rgb(252,78,42)','rgb(253,141,60)','rgb(254,178,76)','rgb(254,217,118)','rgb(255,237,160)','rgb(255,237,160)','rgb(255,255,204)','rgb(255,255,229)','rgb(255,255,255)']
color_scale_list = dict(zip(dist_range,colorlist))

In [14]:
#Create a list of layers based on all zipcode with its color scale
layer_list = []
for key in test.keys():
    layer = dict()
    dataname = 'data_' + key + '.json'
    layer['color'] = color_scale_list[test[key]]
    layer['type'] = 'fill'
    layer['source'] = 'https://raw.githubusercontent.com/stella0316/PizzaMap/master/data/Json/' + dataname
    layer['sourcetype'] = 'geojson'
    
    layer_list.append(layer)

In [15]:
MapData = graph_objs.Data([
    graph_objs.Scattermapbox(
        lat=['45.5017'],
        lon=['-73.5673'],
        mode='markers',
        marker = dict(opacity = 0.7,
                size = 2,
                colorscale = color_scale_list,
                colorbar = dict(
                    title = "Density in Distance")
        )
               )
    
    
])


layout = graph_objs.Layout(
    height=600,
    autosize=True,
    hovermode='closest',
    showlegend=False,
    title='<b>New York Pizza Store HeatMap</b>',
    mapbox=dict(
        layers=[i for i in layer_list],
        accesstoken=mapbox_access_token,
        bearing=0,
        center=dict(
            lat=40.75,
            lon=-73.9
        ),
        pitch=0,
        zoom=10,
        style='streets'
    ),
)

fig = dict(data=MapData, layout=layout)
py.iplot(fig, filename='New York Pizza Store HeatMap')